In [5]:
!pip install boto3 chromadb langchain GitPython langchain_community

In [7]:
pip install -U langchain-aws

  Obtaining dependency information for langchain-aws from https://files.pythonhosted.org/packages/f4/b1/87ad8cac6fc11880af657ead90cc2ff96fae5f9b916bec17016099ac3cb5/langchain_aws-0.1.9-py3-none-any.whl.metadata
  Obtaining dependency information for boto3<1.35.0,>=1.34.131 from https://files.pythonhosted.org/packages/7b/26/76ee022975d33c9460ba0b3edefade5569597aa43bfb57c72722b1c5356a/boto3-1.34.140-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.3,>=0.2.6 from https://files.pythonhosted.org/packages/00/eb/4c320b83d05533f09f9c9d79cdcaea362b05974f3393f74077673cbd16b2/langchain_core-0.2.11-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.35.0,>=1.34.140 from https://files.pythonhosted.org/packages/f5/23/91c8b50588470d80317f4afca93d3d542139bdc38ed5ad1b512fba416af3/botocore-1.34.140-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527

In [1]:
pip install -U langchain-community

  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/be/34/9e064763f81811257667a2a6fc83a3b3e679b62a38fdb6a996c6661c7cd2/langchain_community-0.2.6-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/54/11/aa208123e8cc01bef67e6af9942ba94c1009687def8474eb0eff74ea6cf1/SQLAlchemy-2.0.31-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/97/e7/575ca16871071313a7a7a03fa055f0c3d52f77eb8583b373ac17fc87ec15/aiohttp-3.9.5-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain<0.3.0,>=0.2.6 from https://files.pyt

In [1]:
pip install chromadb


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import langchain
import langchain_community

print(langchain.__version__)

print(langchain_community.__version__)

0.2.6
0.2.6


In [13]:
!pip3.11 install GitPython


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip3.11 install --upgrade pip


In [ ]:
!sudo pip install GitPython
!pip install GitPython --user

Password:

In [1]:
# 3. Amazon Bedrock

import boto3
from langchain_aws import ChatBedrock
from langchain_community.embeddings import BedrockEmbeddings

# Set LLM and embeddings
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

model_id = "anthropic.claude-v2"

model = ChatBedrock (
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs
)

embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v1"
)


# 4. RAG over Code example
# Step-by-step workflow of LangChain code over LangChain Github repo and perform RAG over Python code

# Step 1 - Load (Document Loaders)

import os
# To clone the github repo in the Jupyter Notebook
from git import Repo
from langchain_text_splitters import Language
# To load the files
from langchain_community.document_loaders.generic import GenericLoader
# In the Language Parser pass the extension of the programming language you are working with,
# Here we are working with Pyton, if we are working with, Java or C++ we can add that extension as well

# We load the py code using LanguageParser, which will:
# Keep top-level functions and classes together (into a single document)
# Put remaining code into a separate document
# Retains metadata about where each split comes from

from langchain_community.document_loaders.parsers import LanguageParser

repo_path = "langchain_repo"

if not os.path.exists(repo_path):
    repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)
else:
    print(f"The directory '{repo_path}' already exists. Skipping cloning.")

#Load the Data from the Files
loader = GenericLoader.from_filesystem(
    repo_path+"/libs/langchain/langchain",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

#Step 2 — Transform (Text Splitters) - split the dcoument into chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)


#Step 3 — Embed and Store - embeddings model and vector store
# Using open-source vector database Chroma and Amazon Bedrock Titan Embeddings G1 — Text model.

from langchain_community.vectorstores import Chroma

# save to disk
vectorstore_to_disk = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")


#Step 4 — Retrieve and Generate


# load from disk
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# LCEL RAG Chain 🔗
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

retriever = vectorstore.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain Invoke
response = chain.invoke("How can I use LangChain for production?")
print(response)

# Chain Stream
for chunk in chain.stream("How can I use LangChain for production?"):
    print(chunk, end="", flush=True)



The directory 'langchain_repo' already exists. Skipping cloning.
Here are a few ways you can use LangChain for production:

- Build applications using the LangChain Expression Language (LCEL). LCEL provides a declarative way to build production-grade programs that use LLMs. LCEL programs support synchronous, asynchronous, batch, and streaming operations for scalability.

- Use the Runnable and Chain abstractions. Runnables and Chains make it easy to build reusable components that can be composed into larger programs. The abstractions handle things like state management, observability, and composability.

- Leverage agent toolkits. Agent toolkits contain integrations with external resources like APIs, databases, etc. This allows you to build applications that combine LLMs with external data sources and services.

- Evaluate and monitor your systems. LangChain includes tools for evaluating chains and models using datasets. You can also integrate with LangSmith to evaluate systems and mon